In [1]:
import pandas as pd

In [2]:
data_demo = pd.read_csv("data/demographics_1.csv")

In [3]:
data_demo.head()

,PostCode,Ethnicity,Religion,Language,Immigration,Commute,Marital Status,Family Size,Dwelling,Ownership,Education,Profession,Income,Family Income,Age,Gender
0,E1V6V5,North American,Christian,English,Non-immigrants,"Car, truck, van - as a driver",Married,1 person,Single-detached house,Owned,Postsecondary,62 Health care and social assistance,34560,150658.0,25 to 29 years,Female
1,E1V6V5,British,Christian,French,Non-immigrants,"Car, truck, van - as a driver",Married,2 persons,Single-detached house,Owned,Postsecondary,"56 Administrative and support, waste managemen...",57785,78950.0,45 to 49 years,Male
2,A0A0B7,British,Christian,English and non-official language,Non-immigrants,"Car, truck, van - as a driver",Married,1 person,Single-detached house,Owned,College,31-33 Manufacturing,44055,111262.0,30 to 34 years,Female
3,A0A0B7,North American,Christian,English and non-official language,Non-immigrants,"Car, truck, van - as a driver",Married,2 persons,Single-detached house,Owned,Postsecondary,44-45 Retail trade,35954,64925.0,35 to 39 years,Male
4,E4T0C1,North American,Christian,French,Non-immigrants,"Car, truck, van - as a driver",Married,2 persons,Single-detached house,Owned,Secondary school,"21 Mining, quarrying, and oil and gas extraction",59829,154926.0,60 to 64 years,Female


### Select features that are potentially correlated with the feature (number of trades) from the aggregated data, and are easily aggregated as well (nominal features that only have two categories).
The candadites are:
1. Immigration
2. Family Size
3. Ownership
4. Dwelling
5. Education
6. Income
7. Family Income
8. Age
9. Gender

First, convert Family Size column to ordinal data.

In [4]:
data_demo["Family Size"] = data_demo["Family Size"].str.split(" ", expand=True)[0]

In [5]:
data_demo["Family Size"] = pd.to_numeric(data_demo["Family Size"], errors="coerce")

In [6]:
data_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29115181 entries, 0 to 29115180
Data columns (total 16 columns):
PostCode          object
Ethnicity         object
Religion          object
Language          object
Immigration       object
Commute           object
Marital Status    object
Family Size       int64
Dwelling          object
Ownership         object
Education         object
Profession        object
Income            int64
Family Income     float64
Age               object
Gender            object
dtypes: float64(1), int64(2), object(13)
memory usage: 3.5+ GB


In [7]:
removed_col = ["Ethnicity", "Religion", "Language", "Commute", "Marital Status", "Profession"]
data_demo.drop(removed_col, inplace=True, axis=1)

In [8]:
data_demo.head()

,PostCode,Immigration,Family Size,Dwelling,Ownership,Education,Income,Family Income,Age,Gender
0,E1V6V5,Non-immigrants,1,Single-detached house,Owned,Postsecondary,34560,150658.0,25 to 29 years,Female
1,E1V6V5,Non-immigrants,2,Single-detached house,Owned,Postsecondary,57785,78950.0,45 to 49 years,Male
2,A0A0B7,Non-immigrants,1,Single-detached house,Owned,College,44055,111262.0,30 to 34 years,Female
3,A0A0B7,Non-immigrants,2,Single-detached house,Owned,Postsecondary,35954,64925.0,35 to 39 years,Male
4,E4T0C1,Non-immigrants,2,Single-detached house,Owned,Secondary school,59829,154926.0,60 to 64 years,Female


In [17]:
def aggregate_group(x):
    d = {}
    d["Immigrants Precentage"] = x["Immigration"].value_counts()["Immigrants"] / x["Immigration"].count()\
        if "Immigrants" in x["Immigration"].value_counts() else 0
    return pd.Series(d)

agg_df = data_demo.groupby("PostCode").apply(aggregate_group)

In [18]:
agg_df.head()

,Immigrants Precentage
PostCode,
A0A0B7,0.000000
A0A0C2,0.000000
A0A1A0,0.049180
A0A1B0,0.010417
A0A1C0,0.009166
